In [88]:
%load_ext autoreload
%autoreload 2
import transformers
import torch
import math
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from mcllm.model.llm import *
from tqdm import tqdm
import torch.utils.data as data
from mcllm.data.synthetic import LowRankDataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [89]:
# single prediction
seq_len = 100
batch_size = 1
x = torch.Tensor(np.zeros((batch_size, seq_len)))

llm = NanoBERT(dropout=0, n_embed=10)  # max_seq_len=seq_len)

out = llm(x)
print('shapes', x.shape, out.shape)

TypeError: NanoBERT.forward() missing 1 required positional argument: 'nan_mask'

In [92]:
# training arguments
device = 'cuda'
num_epochs = 100
lr = 1e-3
batch_size = 128
n_embed = 10
dropout = 0

# data arguments a dataloader
n_rows = 8
n_columns = 10
rank = 3
frac_nan_mask = 0.1
seed = 13
n_matrices = 2048

llm = NanoBERT(n_embed=n_embed, n_layers=3, n_heads=3,
               dropout=dropout, n_rows=n_rows, n_columns=n_columns).to(
    device)  # max_seq_len=seq_len)

dataset = LowRankDataset(n_rows, n_columns, rank, frac_nan_mask,
                         seed=seed, length=n_matrices)


llm = llm.to(device)
optimizer = torch.optim.Adam(llm.parameters(), lr=lr)
dataloader = data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
llm.train()
for i in range(num_epochs):
    # print(f'Epoch: {i + 1}')
    train_loss = 0.0
    train_preds = []
    train_labels = []

    llm.train()

    for batch_num, (x_nan_t, x_clean_t, nan_mask_t) in enumerate(dataloader):
        x_nan_t = x_nan_t.to(device)
        x_clean_t = x_clean_t.to(device)
        nan_mask_t = nan_mask_t.to(device)
        # print('shapes', x_t_nan.shape, x_t_clean.shape, nan_mask_t.shape)

        pred = llm(x_nan_t, nan_mask_t)
        nan_loss = F.mse_loss(
            x_clean_t[nan_mask_t.bool()], pred[nan_mask_t.bool()])

        optimizer.zero_grad()
        nan_loss.backward()
        optimizer.step()

        train_loss += nan_loss.item() / nan_mask_t.sum().item()

        # if batch_num % 10 == 0:
        # print(f'Batch: {batch_num}, Loss: {loss.item():0.3f}')

    print(f'{i} -- Train loss {train_loss:0.4f}')

0 -- Train loss 0.0173
1 -- Train loss 0.0158
2 -- Train loss 0.0157
3 -- Train loss 0.0157
4 -- Train loss 0.0157
5 -- Train loss 0.0156
6 -- Train loss 0.0155
7 -- Train loss 0.0159
8 -- Train loss 0.0157
9 -- Train loss 0.0155
10 -- Train loss 0.0157
11 -- Train loss 0.0155
12 -- Train loss 0.0158
13 -- Train loss 0.0157
14 -- Train loss 0.0159
15 -- Train loss 0.0157
16 -- Train loss 0.0158
17 -- Train loss 0.0153
18 -- Train loss 0.0155
19 -- Train loss 0.0158
20 -- Train loss 0.0153
21 -- Train loss 0.0158
22 -- Train loss 0.0158
23 -- Train loss 0.0158
24 -- Train loss 0.0155
